In [54]:
from scrape_billboard import *
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [55]:
#goal = album or track
#send request to lastfm (for album or track data) and return json obj for album or track
def geturl(goal,artist,title):
    url='https://ws.audioscrobbler.com/2.0/?method={}.getinfo&artist={}&api_key=cbe33f95f9dc5877b258dea8d58c9e14&{}&extended=1&format=json'
    artist = artist.replace(' ', '+')
    title = goal+'='+title.replace(' ', '+')
    url=url.format(goal,artist,title)
    rec=None
    try:
        rec=requests.get(url).json()
    except:
        pass
    return rec

In [59]:
#fetch album data and all data for songs on track
def getalbum(artist,title):
    missed = []
    try:
        json=geturl('album',artist,title)
        artist = json['album']['artist']
        album = json['album']['name']
        tracks_data = json['album']['tracks']['track']
        tracks_row = [[track['name'], track['duration'], track['@attr']['rank'], getsong(artist, track['name'])] for track in tracks_data]
        row_dict = []
        for row in tracks_row:
            row_dict.append({'artist': artist, 'album': album, 'track-title':row[0], 'duration': row[1], 
                            'track_no': row[2], 'listeners': row[3][0], 'playcount': row[3][1], 'single_release': row[3][2]})
        tracklist = pd.DataFrame(row_dict)
        return tracklist
    except:
        #log missed albums
        missed.append((artist, title))
        

In [60]:
#get single song data
def getsong(artist,title):
    trackrelease=None
    try:
        song=geturl('track',artist,title)
    except:
        pass
    try:
        listeners=(song['track']['listeners'])
    except:
        pass
    try:
        playcount=(song['track']['playcount'])
    except:
        pass
    try:
        trackrelease=(song['track']['wiki']['published'])
    except:
        pass 
    return listeners, playcount,trackrelease

In [92]:
#albums to scrape
albums2018 = scrape_billboard_albums(billboard_url)

In [63]:
df=pd.DataFrame()
for album in albums2018:
    project = getalbum(album[0],album[1])
    df = pd.concat([df,project])

Khalid American Teen
Juice WRLD Goodbye %26 Good Riddance
Luke Combs This One's For You
Kane Brown Kane Brown
G-Eazy The Beautiful %26 Damned
Chris Brown Heartbreak On A Full Moon
Drake Views
Rae Sremmurd SR3MM
Miguel War %27 Leisure
Wiz Khalifa Rolling Papers 2
The Chainsmokers Memories%2E%2E%2EDo Not Open
DJ Khaled Grateful
Trippie Redd A Love Letter To You
Drake If you're reading this it's too late
Charlie Puth Voicenotes
U2 Songs Of Experience


In [65]:
#write to csv
df.to_csv('./lastfm.csv')